In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

from tabulate import tabulate


print("--")


In [ ]:

import dask.dataframe as dd
import numpy as np
import pandas as pd
   #
import json

from tabulate import tabulate

print("--")


In [ ]:

#  Load DataFrame with raw input data

l_InputFiles  = [
   "/home/jovyan/work/My_KG_NoteBooks/N4_GNN_MovieDB/02_Files/50_Keywords_00.txt",
]

dd_data = dd.read_csv(
   l_InputFiles,
   delimiter  = ",",
   skiprows   = 1,                                            #  Skip the first line of each file, since it's the column headers
   dtype      = {
      "id"                        : np.dtype(str),
      "keywords"                  : np.dtype(str),            #  In the source CSV, this column was titled 'values', a bad idea
      },
   names      = [
      "id", "keywords"
      ]
   )   

dd_data2  = dd_data[["id"]]
dd_data3  = dd_data2.assign(LABEL=lambda x: "Keyword")
dd_data3.compute()

print("--")


In [ ]:
# ##########################################################

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:
#  CREATE GRAPHS

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)
print("")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:

l_result = my_graph.query("""

   MATCH (n) -[r]-> (m)
   RETURN n, r, m
   
   """,
   contextualize=True)

l_result.view()


In [ ]:
# ##########################################################

In [ ]:


pd_data   = dd_data3.compute()
print(type(pd_data))
print(pd_data.columns)
print("")


da_data      = dd_data3.to_dask_array()
   #
dd_data4     = dd.io.from_dask_array(da_data,   columns=[
   "id", "LABEL"
   ])
dd_data4.compute()
   #
print(dd_data4._meta.columns)
print("")


dd_data8 = dd.from_pandas(pd_data, npartitions=10)
dd_data8.compute()
   #
print(dd_data8._meta.columns)
print("")


In [ ]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   #  Keywords
   #
   df_importer.nodes_dataframe(
       
      dd_data4[["id", "LABEL"]],
       
      id_column  = "id",
      id_space   = "Keywords"
      )  
    
print("--")

#  FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/My_KG_NoteBooks/N4_GNN_MovieDB/02_Files/50_Keywords_00.txt'


In [116]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   #  Keywords
   #
   df_importer.nodes_dataframe(
       
      dd_data8[["id", "LABEL"]],
       
      id_column  = "id",
      id_space   = "Keywords"
      )  
    
print("--")


          0/? [?op/s]

ValueError: You must declare the operation to perform, one of: insert(), update(), upsert(), delete()